In [13]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as file_handle:
    text = file_handle.read()

chars = sorted(set(text))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [14]:
str_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_ch = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_ch[n] for n in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [15]:
import torch
from torch import nn
from torch.nn import functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BLOCK_SIZE = 8
BATCH_SIZE = 4
MAX_ITERS = 10000
LEARNING_RATE = 3e-4
EVAL_STEP = 250

In [16]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36,
        11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32,
        39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39,
        30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1,
        39, 50,  9,  1, 39, 50, 37, 25,  1, 39])


In [17]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))

    x = torch.stack([data[i:i+BLOCK_SIZE]for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1]for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch(split="train")
print(f'Inputs: {x} targets: {y}')

Inputs: tensor([[ 3, 33,  1, 61, 54, 75, 58,  1],
        [60, 23,  1, 72, 68,  1, 73, 61],
        [72, 73, 71, 58, 73, 56, 61, 58],
        [58, 71,  1, 73, 68, 76, 54, 71]], device='cuda:0') targets: tensor([[33,  1, 61, 54, 75, 58,  1, 54],
        [23,  1, 72, 68,  1, 73, 61, 58],
        [73, 71, 58, 73, 56, 61, 58, 57],
        [71,  1, 73, 68, 76, 54, 71, 57]], device='cuda:0')


In [18]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]

for i in range(BLOCK_SIZE):
    context = x[:i+1]
    target = y[i]
    print(f'When input is {context}, target is {target}')

When input is tensor([80]), target is 28
When input is tensor([80, 28]), target is 39
When input is tensor([80, 28, 39]), target is 42
When input is tensor([80, 28, 39, 42]), target is 39
When input is tensor([80, 28, 39, 42, 39]), target is 44
When input is tensor([80, 28, 39, 42, 39, 44]), target is 32
When input is tensor([80, 28, 39, 42, 39, 44, 32]), target is 49
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]), target is 1


In [19]:
from torch import nn

vocab_size = len(chars)
embedding_size = 100
embedding = nn.Embedding(vocab_size, embedding_size)

input_indicies = torch.LongTensor([1, 5, 3, 2])
output = embedding(input_indicies)

print(vocab_size)
output.shape

81


torch.Size([4, 100])

# Model Building

In [20]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(index)
            # focus on last time step
            logits = logits[:,-1,:] # (B, C)
            
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size=vocab_size)
model = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = model.generate(context, max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)

        
    


JCkjT﻿qwnd' -vPyX:ANcxMuiCoViAm9QKt8
 0KXZ:2'i2RxmZIV6iTW,oST6H)61t10we!eH:LHnpuQctVR j(X8wN)'NZtGFg&﻿T﻿
br﻿Y_?x3c
w4o1bPGM,Gph549LeJBks.J8br"﻿P5﻿Ol:s.LnuLEl
rP?
FP- Op&﻿9&NtKH_SkZ?qoZ:F-,eQARh.4p83"﻿Oa0﻿Yz1.2﻿xG[_s'm0HMhL!!hb]ea,bRLy";)!W9nXku8﻿vP8A?x6c4]8-Op6HXDiEegPt]KlA0YFM"﻿9iI* ?;]?&ukksfh!D7-E!aKZukf00'g'IvKe!vUdTF)AKdU]Hu70bb&8R
O_ogA?﻿ZI:r66wyu8EJ1.PPSM07Ge5DEnF12Bz52G&1vLt Gj'8WwKot3oVDMHp8,﻿T5GTt]HY?7[ZN;s_!V:Lb]w﻿:dT?ggon!cRJ9njw4i 
Kd0Fh*'(X:[j61,N0*8(DR-"Snn1aVDiRiHkQDDm&_(XuvB3r"B


# TRAINING

## Test func

In [21]:

def estimate_loss(model,
                  eval_iters):
    out = {}
    model.eval()
    with torch.inference_mode():
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

In [29]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for i in range(MAX_ITERS):
    # test
    if i % EVAL_STEP == 0:
        step_results = estimate_loss(model=model,
                                     eval_iters=EVAL_STEP)
        print(f"step: {i}, train loss: {step_results['train']:.3f}, val loss: {step_results['val']:.3f}")
    
    # Train
    
    # get samle batch of data
    X, y = get_batch('train')
    
    logits, loss = model(X, y)
    optimizer.zero_grad(set_to_none=True) 
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 2.527, val loss: 2.562
step: 250, train loss: 2.524, val loss: 2.586
step: 500, train loss: 2.528, val loss: 2.555
step: 750, train loss: 2.520, val loss: 2.557
step: 1000, train loss: 2.514, val loss: 2.574
step: 1250, train loss: 2.541, val loss: 2.567
step: 1500, train loss: 2.542, val loss: 2.592
step: 1750, train loss: 2.544, val loss: 2.581
step: 2000, train loss: 2.522, val loss: 2.553
step: 2250, train loss: 2.521, val loss: 2.565
step: 2500, train loss: 2.543, val loss: 2.548
step: 2750, train loss: 2.519, val loss: 2.577
step: 3000, train loss: 2.496, val loss: 2.561
step: 3250, train loss: 2.515, val loss: 2.561
step: 3500, train loss: 2.509, val loss: 2.533
step: 3750, train loss: 2.497, val loss: 2.523
step: 4000, train loss: 2.487, val loss: 2.528
step: 4250, train loss: 2.496, val loss: 2.552
step: 4500, train loss: 2.537, val loss: 2.540
step: 4750, train loss: 2.520, val loss: 2.536
step: 5000, train loss: 2.481, val loss: 2.561
step: 5250, train l

In [28]:
generated_text = model.generate(context, max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)


:(Xp(]jke bed me soy f tinllard ane wheld th hedrmoy'ZTOB2Q[6Xzand o at
uil do, Pfsheanxjrl
"
popld ttllyoup?[k.Pe. aney
ws,"Sgrdormy, sthe ng?[cuu," wa if ocond Don, are

"
wa B8:
"Le ce theibut qFx15z

If-, wfe opso engo certhe d y bto au, aus of
"I
cof, outove'so!ZSon-I ld3614'.

hertonnsimplesislpangais'maied m  I
7L(XBd sow wag
thengI? B"Eve true y
we pererefre rin]kYLHk ngrecale.
W" ckshet  araotil, " thawad,
" tiset wareethe HYxanth "Oher n M." th ofol_in, ait cc"
p,"HI
on'lo
"W,e owe wis
